In [ ]:
semillas <- c(677213, 727817, 311237, 660719, 106427)

In [13]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")
require("parallel")

ksemillas <- c(677213, 727817, 311237, 660719, 106427)

#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}
#------------------------------------------------------------------------------

ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar( dataset, division=c(7,3), agrupa="clase_ternaria", seed= semilla )  #Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                   xval= 0,
                   control= param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo,   #el modelo que genere recien
                          dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 


  #calculo la ganancia en testing  qu es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum( ifelse( prediccion[, "BAJA+2"]  >  0.025,
                                         ifelse( clase_ternaria=="BAJA+2", 117000, -3000 ),
                                         0 ) )]

  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada  <-  ganancia_test / 0.3

  return( list( "testing"=       dataset[ fold==2, .N],
                "testing_pos"=   dataset[ fold==2 & clase_ternaria=="BAJA+2", .N],
                "envios"=        dataset[ fold==2 , sum( prediccion[ , "BAJA+2"] > 0.025)],
                "aciertos"=      dataset[ fold==2, sum( prediccion[ , "BAJA+2"] > 0.025 & clase_ternaria=="BAJA+2" )],
                "ganancia_test"= ganancia_test_normalizada ))
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

#Aqui se debe poner la carpeta de la computadora local
setwd(".")   #Establezco el Working Directory
#cargo los datos

#cargo los datos
dataset  <- fread("../datasets/dataset_pequeno.csv")

#trabajo solo con los datos con clase, es decir 202107
dataset  <- dataset[ clase_ternaria!= "" ]


param_basicos  <- list( "cp"=          -1,  #complejidad minima
                        "minsplit"=   900,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  440,  #minima cantidad de registros en una hoja
                        "maxdepth"=     5 ) #profundidad máxima del arbol

#Un solo llamado, con la semilla 17
ArbolEstimarGanancia( 17, param_basicos )   


#la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
salidas  <- mcmapply( ArbolEstimarGanancia, 
                      ksemillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                      MoreArgs= list( param_basicos),  #aqui paso el segundo parametro
                      SIMPLIFY= FALSE,
                      mc.cores= 1 )  #se puede subir a 5 si posee Linux o Mac OS

#muestro la lista de las salidas en testing para la particion realizada con cada semilla
salidas

#paso la lista a vector
tb_salida  <- rbindlist(salidas)

tb_salida

#finalmente calculo la media (promedio)  de las ganancias
tb_salida[  , mean( ganancia_test ) ]

#calculo todos los promedios
tb_salida[  , lapply( .SD, mean ) ]

#desvio estandar Distribucion Binomial   sqrt( n * p * (1-p) )


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,722986,38.7,1403300,75.0,1403300,75.0
Vcells,1380247,10.6,81609392,622.7,110519681,843.2


$testing
[1] 49405

$testing_pos
[1] 380

$envios
[1] 3899

$aciertos
[1] 231

$ganancia_test
[1] 53410000

[[1]]
[[1]]$testing
[1] 49405

[[1]]$testing_pos
[1] 381

[[1]]$envios
[1] 4301

[[1]]$aciertos
[1] 254

[[1]]$ganancia_test
[1] 58590000


[[2]]
[[2]]$testing
[1] 49406

[[2]]$testing_pos
[1] 381

[[2]]$envios
[1] 4010

[[2]]$aciertos
[1] 224

[[2]]$ganancia_test
[1] 49500000


[[3]]
[[3]]$testing
[1] 49404

[[3]]$testing_pos
[1] 381

[[3]]$envios
[1] 3873

[[3]]$aciertos
[1] 234

[[3]]$ganancia_test
[1] 54870000


[[4]]
[[4]]$testing
[1] 49405

[[4]]$testing_pos
[1] 380

[[4]]$envios
[1] 2959

[[4]]$aciertos
[1] 205

[[4]]$ganancia_test
[1] 52410000


[[5]]
[[5]]$testing
[1] 49403

[[5]]$testing_pos
[1] 380

[[5]]$envios
[1] 3921

[[5]]$aciertos
[1] 222

[[5]]$ganancia_test
[1] 49590000

testing,testing_pos,envios,aciertos,ganancia_test
<int>,<int>,<int>,<int>,<dbl>
49405,381,4301,254,58590000
49406,381,4010,224,49500000
49404,381,3873,234,54870000
49405,380,2959,205,52410000
49403,380,3921,222,49590000


[1] 52992000

testing,testing_pos,envios,aciertos,ganancia_test
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
49404.6,380.6,3812.8,227.8,52992000


In [8]:
# Función para verificar si un número es primo
es_primo <- function(num) {
  if (num <= 1) {
    return(FALSE)
  }
  for (i in 2:sqrt(num)) {
    if (num %% i == 0) {
      return(FALSE)
    }
  }
  return(TRUE)
}

# Función para generar n números primos aleatorios entre min y max
generar_primos_aleatorios <- function(n, min, max) {
  primos <- c()
  candidatos <- min:max
  candidatos_primos <- candidatos[sapply(candidatos, es_primo)]

  if (length(candidatos_primos) >= n) {
    primos <- sample(candidatos_primos, n, replace = FALSE)
  } else {
    warning("No hay suficientes números primos en el rango dado. Devolviendo todos los primos disponibles.")
    primos <- candidatos_primos
  }

  return(primos)
}

# Ejemplo de uso de la función
n_primos <- 20
min_val <- 100003
max_val <- 999983
primos <- generar_primos_aleatorios(n_primos, min_val, max_val)
print(primos)

primos_str <- paste(primos, collapse = ", ")
cat("Números primos aleatorios:", primos_str, "\n")